# SSD Evaluation Tutorial

This is a brief tutorial that explains how compute the average precisions for any trained SSD model using the `Evaluator` class. The `Evaluator` computes the average precisions according to the Pascal VOC pre-2010 or post-2010 detection evaluation algorithms. You can find details about these computation methods [here](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/htmldoc/devkit_doc.html#sec:ap).

As an example we'll evaluate an SSD300 on the Pascal VOC 2007 `test` dataset, but note that the `Evaluator` works for any SSD model and any dataset that is compatible with the `DataGenerator`. If you would like to run the evaluation on a different model and/or dataset, the procedure is analogous to what is shown below, you just have to build the appropriate model and load the relevant dataset.

Note: I that in case you would like to evaluate a model on MS COCO, I would recommend to follow the [MS COCO evaluation notebook](https://github.com/pierluigiferrari/ssd_keras/blob/master/ssd300_evaluation_COCO.ipynb) instead, because it can produce the results format required by the MS COCO evaluation server and uses the official MS COCO evaluation code, which computes the mAP slightly differently from the Pascal VOC method.

Note: In case you want to evaluate any of the provided trained models, make sure that you build the respective model with the correct set of scaling factors to reproduce the official results. The models that were trained on MS COCO and fine-tuned on Pascal VOC require the MS COCO scaling factors, not the Pascal VOC scaling factors.

In [1]:
from keras import backend as K
from keras.models import load_model
from keras.optimizers import Adam
from scipy.misc import imread
import numpy as np
from matplotlib import pyplot as plt

from models.keras_ssd300 import ssd_300
from keras_loss_function.keras_ssd_loss_mod import SSDLoss
from keras_loss_function.keras_ssd_loss_proj import SSDLoss_proj
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization
from data_generator.object_detection_2d_data_generator import DataGenerator
from eval_utils.average_precision_evaluator import Evaluator

%matplotlib inline

Using TensorFlow backend.


In [2]:
# Set a few configuration parameters.
img_height = 300
img_width = 600
n_classes = 1
model_mode = 'inference'

## 1. Load a trained SSD

Either load a trained model or build a model and load trained weights into it. Since the HDF5 files I'm providing contain only the weights for the various SSD versions, not the complete models, you'll have to go with the latter option when using this implementation for the first time. You can then of course save the model and next time load the full model directly, without having to build it.

You can find the download links to all the trained model weights in the README.

### 1.1. Build the model and load trained weights into it

In [3]:
# 1: Build the Keras model

K.clear_session() # Clear previous models from memory.

model = ssd_300(image_size=(img_height, img_width, 3),
                n_classes=n_classes,
                mode=model_mode,
                l2_regularization=0.0005,
                scales=[0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05], # The scales for MS COCO [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05]
                aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5]],
                two_boxes_for_ar1=True,
                steps=[8, 16, 32, 64, 100, 300],
                offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
                clip_boxes=False,
                variances=[0.1, 0.1, 0.2, 0.2],
                normalize_coords=True,
                subtract_mean=[123, 117, 104],
                swap_channels=[2, 1, 0],
                confidence_thresh=0.01,
                iou_threshold=0.5,
                top_k=200,
                nms_max_output_size=400)

# 2: Load the trained weights into the model.

# TODO: Set the path of the trained weights.
weights_path = 'checkpoints/EXP 1/double_ssd300_pascal_07+12_epoch-10_loss-69.1002_val_loss-63.0645.h5'

model.load_weights(weights_path, by_name=True)

# 3: Compile the model so that Keras won't complain the next time you load it.

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

/home/nassar/gits/ssd_keras_double/models/keras_ssd300.py:612: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=[<tf.Tenso...)`
  model = Model(input=[x,geo_1,geo_2],output=[predictions, predictions_proj])


Tensor("decoded_predictions/loop_over_batch/TensorArrayStack/TensorArrayGatherV3:0", shape=(?, ?, ?), dtype=float32)


Or

### 1.2. Load a trained model

We set `model_mode` to 'inference' above, so the evaluator expects that you load a model that was built in 'inference' mode. If you're loading a model that was built in 'training' mode, change the `model_mode` parameter accordingly.

## 2. Create a data generator for the evaluation dataset

Instantiate a `DataGenerator` that will serve the evaluation dataset during the prediction phase.

In [ ]:
model.summary()

In [4]:
dataset = DataGenerator()

# TODO: Set the paths to the dataset here.
VOC_2007_images_dir      = '../datasets/Images/'
VOC_2007_annotations_dir      = '../datasets/VOC/Pasadena/Annotations/'
VOC_2007_test_image_set_filename     = '../datasets/VOC/Pasadena/ImageSets/Main/siamese/val_sia.txt'

# The XML parser needs to now what object class names to look for and in which order to map them to integers.
classes = ['background',
           'tree']

dataset.parse_xml(images_dirs=[VOC_2007_images_dir],
                  image_set_filenames=[VOC_2007_test_image_set_filename],
                  annotations_dirs=[VOC_2007_annotations_dir],
                  classes=classes,
                  include_classes='all',
                  exclude_truncated=False,
                  exclude_difficult=False,
                  ret=False)

Processing image set 'val_sia.txt': 100%|██████████| 3601/3601 [04:30<00:00, 13.33it/s]


## 3. Run the evaluation

Now that we have instantiated a model and a data generator to serve the dataset, we can set up the evaluator and run the evaluation.

The evaluator is quite flexible: It can compute the average precisions according to the Pascal VOC pre-2010 algorithm, which samples 11 equidistant points of the precision-recall curves, or according to the Pascal VOC post-2010 algorithm, which integrates numerically over the entire precision-recall curves instead of sampling a few individual points. You could also change the number of sampled recall points or the required IoU overlap for a prediction to be considered a true positive, among other things. Check out the `Evaluator`'s documentation for details on all the arguments.

In its default settings, the evaluator's algorithm is identical to the official Pascal VOC pre-2010 Matlab detection evaluation algorithm, so you don't really need to tweak anything unless you want to.

The evaluator roughly performs the following steps: It runs predictions over the entire given dataset, then it matches these predictions to the ground truth boxes, then it computes the precision-recall curves for each class, then it samples 11 equidistant points from these precision-recall curves to compute the average precision for each class, and finally it computes the mean average precision over all classes.

In [5]:
evaluator = Evaluator(model=model,
                      n_classes=n_classes,
                      data_generator=dataset,
                      model_mode=model_mode)

In [6]:
evaluator.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 300, 600, 3)   0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 300, 600, 3)   0                                            
____________________________________________________________________________________________________
identity_layer__1 (Lambda)       (None, 300, 600, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
identity_layer__2 (Lambda)       (None, 300, 600, 3)   0           input_2[0][0]                    
___________________________________________________________________________________________

In [7]:
results = evaluator(img_height=img_height,
                    img_width=img_width,
                    batch_size=8,
                    data_generator_mode='resize',
                    round_confidences=False,
                    matching_iou_threshold=0.5,
                    border_pixels='include',
                    sorting_algorithm='quicksort',
                    average_precision_mode='sample',
                    num_recall_points=11,
                    ignore_neutral_boxes=True,
                    return_precisions=True,
                    return_recalls=True,
                    return_average_precisions=True,
                    verbose=True)



Number of images in the evaluation dataset: 3601

Producing predictions batch-wise:   0%|          | 0/451 [00:00<?, ?it/s]Test:  (8, 200, 6)
y_pred:  [[[ 1.00000000e+00  1.03603348e-01  5.52601990e+02  2.61204773e+02
    5.98100586e+02  2.98498383e+02]
  [ 1.00000000e+00  1.02438048e-01  5.45768311e+02  2.16513306e+02
    5.90327576e+02  2.54956451e+02]
  [ 1.00000000e+00  1.02278084e-01  1.23071642e+01  5.17744995e+02
    4.98248901e+01  5.48982117e+02]
  ...
  [ 1.00000000e+00  8.72967094e-02  4.83104362e+01  1.35213715e+02
    1.46670731e+02  4.08107056e+02]
  [ 1.00000000e+00  8.72964039e-02  5.28763306e+02  1.85444565e+02
    6.05684753e+02  3.30690979e+02]
  [ 1.00000000e+00  8.72960985e-02  3.90403717e+02 -1.12365828e+01
    4.69519073e+02  7.42207870e+01]]

 [[ 1.00000000e+00  1.03603348e-01  5.52601990e+02  2.61204773e+02
    5.98100586e+02  2.98498383e+02]
  [ 1.00000000e+00  1.02438048e-01  5.45768311e+02  2.16513306e+02
    5.90327576e+02  2.54956451e+02]
  [ 1.00000000e+0

Producing predictions batch-wise:   1%|          | 3/451 [00:05<15:43,  2.11s/it]Test:  (8, 200, 6)
y_pred:  [[[ 1.00000000e+00  1.03603348e-01  5.52601990e+02  2.61204773e+02
    5.98100586e+02  2.98498383e+02]
  [ 1.00000000e+00  1.02438048e-01  5.45768311e+02  2.16513306e+02
    5.90327576e+02  2.54956451e+02]
  [ 1.00000000e+00  1.02278084e-01  1.23071642e+01  5.17744995e+02
    4.98248901e+01  5.48982117e+02]
  ...
  [ 1.00000000e+00  8.72967094e-02  4.83104362e+01  1.35213715e+02
    1.46670731e+02  4.08107056e+02]
  [ 1.00000000e+00  8.72964039e-02  5.28763306e+02  1.85444565e+02
    6.05684753e+02  3.30690979e+02]
  [ 1.00000000e+00  8.72960985e-02  3.90403717e+02 -1.12365828e+01
    4.69519073e+02  7.42207870e+01]]

 [[ 1.00000000e+00  1.03603348e-01  5.52601990e+02  2.61204773e+02
    5.98100586e+02  2.98498383e+02]
  [ 1.00000000e+00  1.02438048e-01  5.45768311e+02  2.16513306e+02
    5.90327576e+02  2.54956451e+02]
  [ 1.00000000e+00  1.02278084e-01  1.23071642e+01  5.17744

Producing predictions batch-wise:   1%|▏         | 6/451 [00:08<10:01,  1.35s/it]Test:  (8, 200, 6)
y_pred:  [[[ 1.00000000e+00  1.03603348e-01  5.52601990e+02  2.61204773e+02
    5.98100586e+02  2.98498383e+02]
  [ 1.00000000e+00  1.02438048e-01  5.45768311e+02  2.16513306e+02
    5.90327576e+02  2.54956451e+02]
  [ 1.00000000e+00  1.02278024e-01  1.23071594e+01  5.17744812e+02
    4.98248940e+01  5.48981873e+02]
  ...
  [ 1.00000000e+00  8.72966796e-02  4.83104362e+01  1.35213745e+02
    1.46670731e+02  4.08107117e+02]
  [ 1.00000000e+00  8.72964039e-02  5.28763306e+02  1.85444565e+02
    6.05684753e+02  3.30690979e+02]
  [ 1.00000000e+00  8.72960985e-02  3.90403717e+02 -1.12365828e+01
    4.69519073e+02  7.42207870e+01]]

 [[ 1.00000000e+00  1.03603348e-01  5.52601990e+02  2.61204773e+02
    5.98100586e+02  2.98498383e+02]
  [ 1.00000000e+00  1.02438048e-01  5.45768311e+02  2.16513306e+02
    5.90327576e+02  2.54956451e+02]
  [ 1.00000000e+00  1.02278002e-01  1.23072758e+01  5.17745

Producing predictions batch-wise:   2%|▏         | 9/451 [00:11<08:21,  1.13s/it]Test:  (8, 200, 6)
y_pred:  [[[ 1.0000000e+00  1.0360335e-01  5.5260199e+02  2.6120477e+02
    5.9810059e+02  2.9849838e+02]
  [ 1.0000000e+00  1.0243805e-01  5.4576831e+02  2.1651331e+02
    5.9032758e+02  2.5495645e+02]
  [ 1.0000000e+00  1.0227808e-01  1.2307164e+01  5.1774500e+02
    4.9824890e+01  5.4898212e+02]
  ...
  [ 1.0000000e+00  8.7296709e-02  4.8310436e+01  1.3521371e+02
    1.4667073e+02  4.0810706e+02]
  [ 1.0000000e+00  8.7296404e-02  5.2876331e+02  1.8544456e+02
    6.0568475e+02  3.3069098e+02]
  [ 1.0000000e+00  8.7296098e-02  3.9040372e+02 -1.1236583e+01
    4.6951907e+02  7.4220787e+01]]

 [[ 1.0000000e+00  1.0360335e-01  5.5260199e+02  2.6120477e+02
    5.9810059e+02  2.9849838e+02]
  [ 1.0000000e+00  1.0243805e-01  5.4576831e+02  2.1651331e+02
    5.9032758e+02  2.5495645e+02]
  [ 1.0000000e+00  1.0227808e-01  1.2307164e+01  5.1774500e+02
    4.9824890e+01  5.4898212e+02]
  ...
  [ 

Producing predictions batch-wise:   3%|▎         | 12/451 [00:14<07:43,  1.05s/it]Test:  (8, 200, 6)
y_pred:  [[[ 1.00000000e+00  1.03603348e-01  5.52601990e+02  2.61204773e+02
    5.98100586e+02  2.98498383e+02]
  [ 1.00000000e+00  1.02438048e-01  5.45768311e+02  2.16513306e+02
    5.90327576e+02  2.54956451e+02]
  [ 1.00000000e+00  1.02278084e-01  1.23071642e+01  5.17744995e+02
    4.98248901e+01  5.48982117e+02]
  ...
  [ 1.00000000e+00  8.72964039e-02  5.28763306e+02  1.85444565e+02
    6.05684753e+02  3.30690979e+02]
  [ 1.00000000e+00  8.72960985e-02  3.90403717e+02 -1.12365828e+01
    4.69519073e+02  7.42207870e+01]
  [ 1.00000000e+00  8.72959197e-02  5.89842529e+02 -1.86964386e+02
    7.45741821e+02  1.16848396e+02]]

 [[ 1.00000000e+00  1.03603348e-01  5.52601990e+02  2.61204773e+02
    5.98100586e+02  2.98498383e+02]
  [ 1.00000000e+00  1.02438048e-01  5.45768311e+02  2.16513306e+02
    5.90327576e+02  2.54956451e+02]
  [ 1.00000000e+00  1.02278084e-01  1.23071642e+01  5.1774

...

Producing predictions batch-wise:  98%|█████████▊| 444/451 [07:20<00:07,  1.03s/it]Test:  (8, 200, 6)
y_pred:  [[[ 1.00000000e+00  1.03603348e-01  5.52601990e+02  2.61204773e+02
    5.98100586e+02  2.98498383e+02]
  [ 1.00000000e+00  1.02438048e-01  5.45768311e+02  2.16513306e+02
    5.90327576e+02  2.54956451e+02]
  [ 1.00000000e+00  1.02278084e-01  1.23071642e+01  5.17744995e+02
    4.98248901e+01  5.48982117e+02]
  ...
  [ 1.00000000e+00  8.73004794e-02  5.27973022e+02  1.24372498e+02
    6.06286377e+02  2.62669189e+02]
  [ 1.00000000e+00  8.72967094e-02  4.83104362e+01  1.35213745e+02
    1.46670731e+02  4.08107117e+02]
  [ 1.00000000e+00  8.72964039e-02  5.28763306e+02  1.85444565e+02
    6.05684753e+02  3.30690979e+02]]

 [[ 1.00000000e+00  1.03603348e-01  5.52601990e+02  2.61204773e+02
    5.98100586e+02  2.98498383e+02]
  [ 1.00000000e+00  1.02438048e-01  5.45768311e+02  2.16513306e+02
    5.90327576e+02  2.54956451e+02]
  [ 1.00000000e+00  1.02278084e-01  1.23071642e+01  5.177

Producing predictions batch-wise:  99%|█████████▉| 447/451 [07:23<00:04,  1.03s/it]Test:  (8, 200, 6)
y_pred:  [[[ 1.00000000e+00  1.03603348e-01  5.52601990e+02  2.61204773e+02
    5.98100586e+02  2.98498383e+02]
  [ 1.00000000e+00  1.02438048e-01  5.45768311e+02  2.16513306e+02
    5.90327576e+02  2.54956451e+02]
  [ 1.00000000e+00  1.02278084e-01  1.23071642e+01  5.17744995e+02
    4.98248901e+01  5.48982117e+02]
  ...
  [ 1.00000000e+00  8.72967094e-02  4.83104362e+01  1.35213745e+02
    1.46670731e+02  4.08107117e+02]
  [ 1.00000000e+00  8.72964039e-02  5.28763306e+02  1.85444565e+02
    6.05684753e+02  3.30690979e+02]
  [ 1.00000000e+00  8.72960985e-02  3.90403717e+02 -1.12365828e+01
    4.69519073e+02  7.42207870e+01]]

 [[ 1.00000000e+00  1.03603236e-01  5.52601990e+02  2.61204590e+02
    5.98100586e+02  2.98498199e+02]
  [ 1.00000000e+00  1.02437995e-01  5.45768311e+02  2.16513397e+02
    5.90327576e+02  2.54956543e+02]
  [ 1.00000000e+00  1.02277853e-01  1.23071623e+01  5.177

Producing predictions batch-wise: 100%|█████████▉| 450/451 [07:26<00:00,  1.02it/s]Test:  (1, 200, 6)
y_pred:  [[[ 1.0000000e+00  1.0360335e-01  5.5260199e+02  2.6120477e+02
    5.9810059e+02  2.9849838e+02]
  [ 1.0000000e+00  1.0243805e-01  5.4576831e+02  2.1651331e+02
    5.9032758e+02  2.5495645e+02]
  [ 1.0000000e+00  1.0227808e-01  1.2307164e+01  5.1774500e+02
    4.9824890e+01  5.4898212e+02]
  ...
  [ 1.0000000e+00  8.7296680e-02  4.8310436e+01  1.3521373e+02
    1.4667073e+02  4.0810709e+02]
  [ 1.0000000e+00  8.7296404e-02  5.2876331e+02  1.8544456e+02
    6.0568475e+02  3.3069098e+02]
  [ 1.0000000e+00  8.7296113e-02  3.9040372e+02 -1.1236583e+01
    4.6951907e+02  7.4220787e+01]]]
Producing predictions batch-wise: 100%|██████████| 451/451 [07:26<00:00,  1.20it/s]
predictions:  

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Matching predictions to ground truth, class 1/1.: 100%|██████████| 720200/720200 [01:22<00:00, 8720.11it/s] 
Computing precisions and recalls, class 1/1
Computing average precision, class 1/1


In [ ]:
results

In [ ]:
mean_average_precision, average_precisions, precisions, recalls = results

## 4. Visualize the results

Let's take a look:

In [ ]:
for i in range(1, len(average_precisions)):
    print("{:<14}{:<6}{}".format(classes[i], 'AP', round(average_precisions[i], 3)))
print()
print("{:<14}{:<6}{}".format('','mAP', round(mean_average_precision, 3)))

In [ ]:
m = max((n_classes + 1) // 2, 2)
n = 2

fig, cells = plt.subplots(m, n, figsize=(n*8,m*8))
for i in range(m):
    for j in range(n):
        if n*i+j+1 > n_classes: break
        cells[i, j].plot(recalls[n*i+j+1], precisions[n*i+j+1], color='blue', linewidth=1.0)
        cells[i, j].set_xlabel('recall', fontsize=14)
        cells[i, j].set_ylabel('precision', fontsize=14)
        cells[i, j].grid(True)
        cells[i, j].set_xticks(np.linspace(0,1,11))
        cells[i, j].set_yticks(np.linspace(0,1,11))
        cells[i, j].set_title("{}, AP: {:.3f}".format(classes[n*i+j+1], average_precisions[n*i+j+1]), fontsize=16)


## 5. Advanced use

`Evaluator` objects maintain copies of all relevant intermediate results like predictions, precisions and recalls, etc., so in case you want to experiment with different parameters, e.g. different IoU overlaps, there is no need to compute the predictions all over again every time you make a change to a parameter. Instead, you can only update the computation from the point that is affected onwards.

The evaluator's `__call__()` method is just a convenience wrapper that executes its other methods in the correct order. You could just call any of these other methods individually as shown below (but you have to make sure to call them in the correct order).

Note that the example below uses the same evaluator object as above. Say you wanted to compute the Pascal VOC post-2010 'integrate' version of the average precisions instead of the pre-2010 version computed above. The evaluator object still has an internal copy of all the predictions, and since computing the predictions makes up the vast majority of the overall computation time and since the predictions aren't affected by changing the average precision computation mode, we skip computing the predictions again and instead only compute the steps that come after the prediction phase of the evaluation. We could even skip the matching part, since it isn't affected by changing the average precision mode either. In fact, we would only have to call `compute_average_precisions()` `compute_mean_average_precision()` again, but for the sake of illustration we'll re-do the other computations, too.

In [ ]:
evaluator.get_num_gt_per_class(ignore_neutral_boxes=True,
                               verbose=False,
                               ret=False)

evaluator.match_predictions(ignore_neutral_boxes=True,
                            matching_iou_threshold=0.5,
                            border_pixels='include',
                            sorting_algorithm='quicksort',
                            verbose=True,
                            ret=False)

precisions, recalls = evaluator.compute_precision_recall(verbose=True, ret=True)

average_precisions = evaluator.compute_average_precisions(mode='integrate',
                                                          num_recall_points=11,
                                                          verbose=True,
                                                          ret=True)

mean_average_precision = evaluator.compute_mean_average_precision(ret=True)

In [ ]:
for i in range(1, len(average_precisions)):
    print("{:<14}{:<6}{}".format(classes[i], 'AP', round(average_precisions[i], 3)))
print()
print("{:<14}{:<6}{}".format('','mAP', round(mean_average_precision, 3)))